In [ ]:
#利用Python API，获取GEE平台数据，识别并导出指定区域savanna分布图，运行完提示"Success, result saved to GEE."


#整体由以下几部分构成：
#①（函数load_shapefiles）首先导入研究区shapefile面文件（shpdata），无人机影像分类得到的覆盖度训练（labeldatatrain）和验证数据（labeldatatest），作为覆盖度回归模型训练的标签。
#②（函数preprocess_data）加载SRTM地形数据、Sentinel-2 和 Sentinel-1 影像并选取波段，影像日期可自行修改。计算地形特征、纹理和植被指数，作为覆盖度回归模型训练的特征；
#③（函数create_dataset）将标签数据对应到相应地理位置像元的特征数据；
#④（函数train_model）分别训练木本植物和草本植物覆盖度模型并估算研究区木本植物和草本植物覆盖度；
#⑤（函数predict_evaluate_model）利用RMSE指标验证模型精度；
#⑥（函数identify_savanna）提取符合稀树草原的木本和草本植物覆盖度范围定义的像元，生成稀树草原分布图；
#⑦（函数save_results）结果保存，需要登录GEE查看结果保存进度，登录google cloud下载。


#数据说明：
#①加载的地面训练和验证数据为包含木本植物覆盖度和草本植物覆盖度属性信息的shapefile地理点文件。
#②覆盖度回归模型训练的特征包括 a波段反射率（ Sentinel-1 VV/VH、Sentinel-2 B2/B3/B4/B5/B6/B7/B8/B8A/B11/B12）; b地形特征(海拔/坡向/坡度); c植被指数(ndvi/dvi/gndvi/gri/mtci/evi/lswi/bi)
# d纹理（(Blue-mean/Green-mean/Red-mean/NIR-mean/RedEdge1-mean/RedEdge2-mean/RedEdge1-mean/Purple-mean/Yellow-mean/NIR-var/NIR-homo/NIR-con/NIR-asm/NIR-diss/NIR-cor）。
#数据导出默认分辨率为70m，可调用save_results设置scale修改。


#首次运行需要安装GEE API相关包
# conda install earthengine-api
# conda install geemap -c conda-forge
# conda install gdal

#每次运行需验证GEE Python API，验证GEE账户，每次使用均需要运行ee.Initialize()进行初始化。

#认证成功后，账号信息会保存，如果后续失效，再次认证即可。


# 需要导入的数据
shpdata = 'D:/mapping_data/shp/area.shp'  # 研究区shapfile面数据
labeldatatrain = r'D:/mapping_data/data/train.shp'  # 训练数据
labeldatatest = r'D:/mapping_data/data/test.shp'  # 测试数据

#定义Sentinel-2 和 Sentinel-1 影像影像日期
start_date = '2022-06-01'
end_date = '2022-09-30'


import ee
import os
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 设置代理
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10809'
geemap.set_proxy(port=10809)

# 初始化Earth Engine
ee.Authenticate()#认证
geemap.ee_initialize()#初始化

# 设置地图
Map = geemap.Map(center=[44.448, 116.550], zoom=3, height=600)

def main():
    """加载 shapefiles 并转换为 Earth Engine 对象。"""
    shp, labeltrain, labeltest = load_shapefiles(shpdata, labeldatatrain, labeldatatest)
    
    """加载 Sentinel-2 和 Sentinel-1 数据并选取波段，计算纹理指数和植被指数。"""
    features, bands = preprocess_data(shp, start_date, end_date)

    """木本植物覆盖度模型的构建、验证"""
    woody_training, woody_testing,woody_validated = create_dataset('woody', features, bands, labeltrain, labeltest)
    woody_model = train_model('woody', woody_training, bands)
    woody_predict = predict_evaluate_model('woody', features, bands, woody_model, woody_validated, return_type='regression')
    
    """草本植物覆盖度模型的构建、验证"""
    herb_training, herb_testing,herb_validated = create_dataset('herb', features, bands, labeltrain, labeltest)
    herb_model = train_model('herb', herb_training, bands)
    herb_predict = predict_evaluate_model('herb', features, bands, herb_model, herb_validated, return_type='regression')
    
    """savanna识别"""
    savanna = identify_savanna(woody_predict, herb_predict,shp)
    
    """结果输出（木本植物覆盖度图、草本植物覆盖度图、稀树草原分布图）"""
    save_results(woody_predict, shp, 'woody_predict', scale = 20)
    save_results(herb_predict, shp, 'herb_predict', scale = 20)
    save_results(savanna, shp, 'savanna')
    print(f"Success, result saved to GEE.")


def load_shapefiles(shpdata, labeldatatrain, labeldatatest):
    """加载 shapefiles 并转换为 Earth Engine 对象。"""
    shp = geemap.shp_to_ee(shpdata)
    labeltrain = geemap.shp_to_ee(labeldatatrain)
    labeltest = geemap.shp_to_ee(labeldatatest)
    return shp, labeltrain, labeltest

def preprocess_data(shp, start_date, end_date):
    """预处理 Sentinel-2 和 Sentinel-1 数据，计算纹理指数和植被指数。"""
    #去云函数
    def cloudfunction_ST2(s2):
        quality = s2.select('QA60')
        RecorteNubes = 1 << 10
        RecorteCirros = 1 << 11
        qa = quality.bitwiseAnd(RecorteNubes).eq(0) and (quality.bitwiseAnd(RecorteCirros).eq(0))
        return s2.updateMask(qa)
    #加载sentinel-2影像
    senl2 = (
        ee.ImageCollection('COPERNICUS/S2_SR')
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
        .map(cloudfunction_ST2)
        .select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12'])
        .median()
        .clipToCollection(shp)
        .round()
    )
    #加载sentinel-1影像
    sen1 = (
        ee.ImageCollection("COPERNICUS/S1_GRD")
        .filterBounds(shp) 
        .filterDate(start_date, end_date)
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) 
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) 
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
    )

    desc = sen1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    asc = sen1.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

    senl1 = ee.Image.cat([
        ee.ImageCollection(asc.select('VH').merge(desc.select('VH'))).mean(),
        ee.ImageCollection(asc.select('VV').merge(desc.select('VV'))).mean(),
    ]).focal_median().clipToCollection(shp)
    
    #加载dem影像
    dem = ee.Image("USGS/SRTMGL1_003")
    elevation = dem.clip(shp)
    topo = ee.Algorithms.Terrain(elevation)
    
    #计算植被指数
    def vegetation_index(blue, green, red, RedEdge1, RedEdge2, RedEdge3, nir, RedEdge4, SWIR1, SWIR2):

        ndvi = senl2.normalizedDifference(['B8','B4']).rename("ndvi")
        dvi_exp = senl2.expression("B8-B4",{"B4":red,"B8":nir}).rename("dvi")
        gndvi_exp = senl2.expression("(B8-B3)/(B8+B3)",{"B3":green,"B8":nir}).rename("gndvi")
        gri_exp = senl2.expression("B8/B3-1",{"B3":green,"B8":nir}).rename("gri")
        mtci_exp = senl2.expression("(B7-B5)/(B6-B4)",{"B4":red,"B5":RedEdge1,"B6":RedEdge2,"B7":RedEdge3}).rename("mtci")
        evi_exp = senl2.expression("2.5*(B8-B4)/B8+6*B4-7.5*B2+1",{"B2":blue,"B4":red,"B8":nir}).rename("evi")
        lswi_exp = senl2.expression("(B8-B11)/(B8+B11)",{"B8":nir,"B11":SWIR1}).rename("lswi")
        bi_exp = senl2.expression("sqrt(B3*B3+B4*B4+B8*B8)",{"B3":green,"B4":red,"B8":nir}).rename("bi")

        vegindex_features = ndvi.addBands([dvi_exp, gndvi_exp, gri_exp, mtci_exp, evi_exp, lswi_exp, bi_exp])
        return vegindex_features
        
    #计算纹理特征
    def texture_index(blue, green, red, RedEdge1, RedEdge2, RedEdge3, nir, RedEdge4, SWIR1, SWIR2):
        glcmblue = blue.glcmTexture(4)
        glcmgreen = green.glcmTexture(4)
        glcmred = red.glcmTexture(4)
        glcmRedEdge1 = RedEdge1.glcmTexture(4)
        glcmRedEdge2 = RedEdge2.glcmTexture(4)
        glcmRedEdge3 = RedEdge3.glcmTexture(4)
        glcmNIR = nir.glcmTexture(4)
        glcmRedEdge4 = RedEdge4.glcmTexture(4)
        glcmSWIR1 = SWIR1.glcmTexture(4)
        glcmSWIR2 = SWIR2.glcmTexture(4)

        B2savg = glcmblue.select('B2_savg')
        B3savg = glcmgreen.select('B3_savg')
        B4savg = glcmred.select('B4_savg')
        B5savg = glcmRedEdge1.select('B5_savg')
        B6savg = glcmRedEdge2.select('B6_savg')
        B7savg = glcmRedEdge3.select('B7_savg')
        B8savg = glcmNIR.select('B8_savg')
        B8Asavg = glcmRedEdge4.select('B8A_savg')
        B11savg = glcmSWIR1.select('B11_savg')
        B12savg = glcmSWIR2.select('B12_savg')

        B8contrast = glcmNIR.select('B8_contrast')
        B8corr = glcmNIR.select('B8_corr')
        B8diss = glcmNIR.select('B8_diss')
        B8asm = glcmNIR.select('B8_asm')
        B8var = glcmNIR.select('B8_var')
        B8idm = glcmNIR.select('B8_idm')
        B8ent = glcmNIR.select('B8_ent')

        texture_features = B2savg.addBands([B3savg, B4savg, B5savg, B6savg, B7savg, B8savg, B8Asavg, B11savg, B12savg, B8contrast, B8corr, B8diss, B8asm, B8var, B8idm, B8ent])
        return texture_features

    blue = senl2.select(['B2']).toUint16()
    green = senl2.select(['B3']).toUint16()
    red = senl2.select(['B4']).toUint16()
    nir = senl2.select(['B8']).toUint16()
    RedEdge1 = senl2.select(['B5']).toUint16()
    RedEdge2 = senl2.select(['B6']).toUint16()
    RedEdge3 = senl2.select(['B7']).toUint16()
    RedEdge4 = senl2.select(['B8A']).toUint16()
    SWIR1 = senl2.select(['B11']).toUint16()
    SWIR2 = senl2.select(['B12']).toUint16()

    veg_index = vegetation_index(blue, green, red, RedEdge1, RedEdge2, RedEdge3, nir, RedEdge4, SWIR1, SWIR2)
    tex_index = texture_index(blue, green, red, RedEdge1, RedEdge2, RedEdge3, nir, RedEdge4, SWIR1, SWIR2)
    topo10 = topo.select(['elevation', 'slope', 'aspect'])  # Assuming topo is the Terrain output with these bands

    #合并特征
    composite_features = senl2.addBands([senl1, veg_index, tex_index, topo10])
    features = composite_features.reproject(crs='EPSG:4326', scale=20)
    bands = features.bandNames()
    
    return features, bands

def create_dataset(vegetation_type, features, bands, labeltrain, labeltest):
    """创建训练和验证数据集"""
    if vegetation_type == 'woody':
        training = features.sampleRegions(
            collection=labeltrain,
            properties=['woody'],
            scale=20,
            geometries=True)
        validateddata = features.sampleRegions(
            collection=labeltest,
            properties=['woody'],
            scale=20,
            geometries=True)
    elif vegetation_type == 'herb':
        training = features.sampleRegions(
            collection=labeltrain,
            properties=['herb'],
            scale=20,
            geometries=True)
        validateddata = features.sampleRegions(
            collection=labeltest,
            properties=['herb'],
            scale=20,
            geometries=True)
    else:
        raise ValueError("Invalid vegetation type. Choose 'woody' or 'herb'.")

    trainingdata = training.randomColumn()
    training = trainingdata.filter(ee.Filter.lt('random', 0.8))
    testing = trainingdata.filter(ee.Filter.gte('random',0.8))
    return training, testing,validateddata

def train_model(vegetation_type, training, bands):
    """随机森林回归"""
    if vegetation_type == 'woody':
        label = 'woody'
    elif vegetation_type == 'herb':
        label = 'herb'
    else:
        raise ValueError("Invalid vegetation type. Choose 'woody' or 'herb'.")
        
    RFmodel = ee.Classifier.smileRandomForest(numberOfTrees=160).setOutputMode('REGRESSION').train(**{
        'features': training,
        'classProperty': label,
        'inputProperties': bands
    })
    return RFmodel

def predict_evaluate_model(vegetation_type, features, bands, RFmodel, validateddata, return_type='regression'):
    """区域覆盖度计算并计算RMSE。"""
    if vegetation_type == 'woody':
        label = 'woody'
    elif vegetation_type == 'herb':
        label = 'herb'
    else:
        raise ValueError("Invalid vegetation type. Choose 'woody' or 'herb'.")
        
    regression = features.select(bands).classify(RFmodel, 'predicted')#研究区覆盖度图生成

    # 计算RMSE
    predictedValidation = regression.sampleRegions(**{
        'collection': validateddata,
        'properties': [label],
        'scale': 20})
    sampleValidation = predictedValidation.select([label, 'predicted'])
    observationValidation = ee.Array(sampleValidation.aggregate_array(label))
    predictionValidation = ee.Array(sampleValidation.aggregate_array('predicted'))
    residualsValidation = observationValidation.subtract(predictionValidation)
    rmseValidation = residualsValidation.pow(2).reduce('mean', [0]).sqrt()
    rmse_value = ee.Number(rmseValidation).getInfo()
    print(f"{label} RMSE:{rmse_value}")
    return regression
       
def save_results(img, shp,description, scale=70):
    """保存结果图像到 Google Drive。"""
    geemap.ee_export_image_to_drive(
        img, 
        description=description, 
        folder=description, 
        region=shp.geometry(), 
        scale=scale, 
        crs='EPSG:4326',
        maxPixels=350000000
    )

def identify_savanna(woodyFVC, herbFVC,shp):
    """识别savanna区域"""
    # 定义识别savanna的条件
    savanna_identify = woodyFVC.gt(10) and woodyFVC.lt(40).And(herbFVC.gt(40))

    # 将条件应用于woody和herb的分类结果
    savanna = woodyFVC.where(savanna_identify, 1).clip(shp)

    return savanna 

if __name__ == "__main__":
    main()